In [8]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread pandas


In [9]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd
from datetime import datetime, timedelta
import gspread


In [10]:
# Google API Configuration
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly', 'https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = r'C:\Users\avine\Downloads\Project\aristok-automation-441704-40507951b10b.json'  # Path to service account JSON file
SEARCH_CONSOLE_SITE_URL = 'https://www.medikabazaar.com/'  # Replace with your site URL
SPREADSHEET_ID = '1G_xGUUl0yGbtkdza9EKD94dcc3LTKO2ADEOY6tdKwME'  # Replace with your Google Sheet ID

# Authenticate Google APIs
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
search_console_service = build('webmasters', 'v3', credentials=credentials)
sheets_client = gspread.authorize(credentials)


In [11]:
def fetch_monthly_data(year, month):
    # Calculate the start and end dates for the month
    start_date = f"{year}-{month:02d}-01"
    end_date = (datetime(year, month + 1, 1) - timedelta(days=1)).strftime('%Y-%m-%d') if month < 12 else f"{year}-12-31"

    # Fetch data
    request = {
        'startDate': start_date,
        'endDate': end_date,
        'dimensions': ['date']
    }
    response = search_console_service.searchanalytics().query(siteUrl=SEARCH_CONSOLE_SITE_URL, body=request).execute()
    rows = response.get('rows', [])

    # Summarize data
    total_clicks = sum(row.get('clicks', 0) for row in rows)
    total_impressions = sum(row.get('impressions', 0) for row in rows)
    avg_ctr = (sum(row.get('ctr', 0) * row.get('impressions', 0) for row in rows) / total_impressions) if total_impressions else 0
    avg_position = (sum(row.get('position', 0) * row.get('impressions', 0) for row in rows) / total_impressions) if total_impressions else 0

    # Return monthly data
    return {
        'Clicks': total_clicks,
        'Impressions': total_impressions,
        'CTR': f"{avg_ctr * 100:.2f}%",
        'Average Position': f"{avg_position:.1f}" if total_impressions else "-"
    }


In [12]:
# Define the year and months to retrieve data for
year = 2024
months = ['Jul', 'Aug', 'Sep', 'Oct']  # Customize the months as needed
month_numbers = [7, 8, 9, 10]  # Corresponding numeric months

# Fetch data for each month and store it in a DataFrame
data = []
for month, month_num in zip(months, month_numbers):
    print(f"Fetching data for {month} {year}...")
    monthly_data = fetch_monthly_data(year, month_num)
    data.append({'Performance': month + f"'{str(year)[-2:]}", **monthly_data})

# Convert to DataFrame
df = pd.DataFrame(data)

# Format for Google Sheets
df['Impressions'] = df['Impressions'].apply(lambda x: f"{x:,}")  # Add commas for thousands
print(df)


Fetching data for Jul 2024...
Fetching data for Aug 2024...
Fetching data for Sep 2024...
Fetching data for Oct 2024...
  Performance  Clicks Impressions    CTR Average Position
0      Jul'24   26033   2,340,879  1.11%             13.3
1      Aug'24   30749   2,968,862  1.04%             13.2
2      Sep'24   34684   3,731,914  0.93%             13.2
3      Oct'24   35267   4,614,220  0.76%             13.4


In [13]:
def export_to_google_sheets(sheet_id, data):
    try:
        # Access the Google Sheet
        sheet = sheets_client.open_by_key(sheet_id)
        worksheet = sheet.sheet1  # Access the first sheet (or use .add_worksheet for new ones)

        # Clear existing data and update with new data
        worksheet.clear()
        worksheet.update([data.columns.values.tolist()] + data.values.tolist())
        print("Data successfully exported to Google Sheets.")
    except Exception as e:
        print(f"Error exporting to Google Sheets: {e}")

# Export the DataFrame to Google Sheets
export_to_google_sheets(SPREADSHEET_ID, df)


Data successfully exported to Google Sheets.
